# Assignment

In [40]:
# Import 

import numpy as np
import matplotlib.pyplot as plt
import torch
from torch import nn
from degree_freedom_queen import *
from degree_freedom_king1 import *
from degree_freedom_king2 import *
from generate_game import *
from Chess_env import *
import torch.nn.functional as F
import random

size_board = 4

## The Environment

You can find the environment in the file Chess_env, which contains the class Chess_env. To define an object, you need to provide the board size considered as input. In our example, size_board=4. 
Chess_env is composed by the following methods:

1. Initialise_game. The method initialises an episode by placing the three pieces considered (Agent's king and queen, enemy's king) in the chess board. The outputs of the method are described below in order.

     S $\;$ A matrix representing the board locations filled with 4 numbers: 0, no piece in that position; 1, location of the 
     agent's king; 2 location of the queen; 3 location of the enemy king.
     
     X $\;$ The features, that is the input to the neural network. See the assignment for more information regarding the            definition of the features adopted. To personalise this, go into the Features method of the class Chess_env() and change        accordingly.
     
     allowed_a $\;$ The allowed actions that the agent can make. The agent is moving a king, with a total number of 8                possible actions, and a queen, with a total number of $(board_{size}-1)\times 8$ actions. The total number of possible actions correspond      to the sum of the two, but not all actions are allowed in a given position (movements to locations outside the borders or      against chess rules). Thus, the variable allowed_a is a vector that is one (zero) for an action that the agent can (can't)      make. Be careful, apply the policy considered on the actions that are allowed only.
     

2. OneStep. The method performs a one step update of the system. Given as input the action selected by the agent, it updates the chess board by performing that action and the response of the enemy king (which is a random allowed action in the settings considered). The first three outputs are the same as for the Initialise_game method, but the variables are computed for the position reached after the update of the system. The fourth and fifth outputs are:

     R $\;$ The reward. To change this, look at the OneStep method of the class where the rewards are set.
     
     Done $\;$ A variable that is 1 if the episode has ended (checkmate or draw).
     
     
3. Features. Given the chessboard position, the method computes the features.

This information and a quick analysis of the class should be all you need to get going. The other functions that the class exploits are uncommented and constitute an example on how not to write a python code. You can take a look at them if you want, but it is not necessary.






In [41]:
## INITIALISE THE ENVIRONMENT

env=Chess_Env(size_board)

In [42]:
## PRINT 5 STEPS OF AN EPISODE CONSIDERING A RANDOM AGENT

S,X,allowed_a=env.Initialise_game()                       # INTIALISE GAME

print(S)                                                  # PRINT CHESS BOARD (SEE THE DESCRIPTION ABOVE)

print('check? ',env.check)                                # PRINT VARIABLE THAT TELLS IF ENEMY KING IS IN CHECK (1) OR NOT (0)
print('dofk2 ',np.sum(env.dfk2_constrain).astype(int))    # PRINT THE NUMBER OF LOCATIONS THAT THE ENEMY KING CAN MOVE TO


for i in range(5):
    
    a,_=np.where(allowed_a==1)                  # FIND WHAT THE ALLOWED ACTIONS ARE

    a_agent=np.random.permutation(a)[0]         # MAKE A RANDOM ACTION


    S,X,allowed_a,R,Done=env.OneStep(a_agent)   # UPDATE THE ENVIRONMENT
    
    
    ## PRINT CHESS BOARD AND VARIABLES
    print('')
    print(S)
    print(R,'', Done)
    print('check? ',env.check)
    print('dofk2 ',np.sum(env.dfk2_constrain).astype(int))
    
    
    # TERMINATE THE EPISODE IF Done=True (DRAW OR CHECKMATE)
    if Done:
        break
        


[[0 0 0 0]
 [0 0 3 0]
 [0 0 0 0]
 [0 0 1 2]]
check?  0
dofk2  2

[[0 3 0 0]
 [0 0 0 0]
 [0 0 2 0]
 [0 0 1 0]]
0  0
check?  0
dofk2  1

[[0 3 0 0]
 [0 0 0 0]
 [0 1 2 0]
 [0 0 0 0]]
0.0  1
check?  0
dofk2  0


In [43]:
# PERFORM N_episodes=1000 EPISODES MAKING RANDOM ACTIONS AND COMPUTE THE AVERAGE REWARD AND NUMBER OF MOVES 

S,X,allowed_a=env.Initialise_game()
N_episodes=1000

# VARIABLES WHERE TO SAVE THE FINAL REWARD IN AN EPISODE AND THE NUMBER OF MOVES 
R_save_random = np.zeros([N_episodes, 1])
N_moves_save_random = np.zeros([N_episodes, 1])

for n in range(N_episodes):
    
    S,X,allowed_a=env.Initialise_game()     # INITIALISE GAME
    Done=0                                  # SET Done=0 AT THE BEGINNING
    i=1                                     # COUNTER FOR THE NUMBER OF ACTIONS (MOVES) IN AN EPISODE
    
    # UNTIL THE EPISODE IS NOT OVER...(Done=0)
    while Done==0:
        
        # SAME AS THE CELL BEFORE, BUT SAVING THE RESULTS WHEN THE EPISODE TERMINATES 
        
        a,_=np.where(allowed_a==1)
        a_agent=np.random.permutation(a)[0]

        S,X,allowed_a,R,Done=env.OneStep(a_agent)
        
        
        if Done:
            
            R_save_random[n]=np.copy(R)
            N_moves_save_random[n]=np.copy(i)

            break

        i=i+1                               # UPDATE THE COUNTER



# AS YOU SEE, THE PERFORMANCE OF A RANDOM AGENT ARE NOT GREAT, SINCE THE MAJORITY OF THE POSITIONS END WITH A DRAW 
# (THE ENEMY KING IS NOT IN CHECK AND CAN'T MOVE)

print('Random_Agent, Average reward:',np.mean(R_save_random),'Number of steps: ',np.mean(N_moves_save_random))



Random_Agent, Average reward: 0.187 Number of steps:  7.157


In [49]:
# INITIALISE THE PARAMETERS OF YOUR NEURAL NETWORK AND...
# PLEASE CONSIDER TO USE A MASK OF ONE FOR THE ACTION MADE AND ZERO OTHERWISE IF YOU ARE NOT USING VANILLA GRADIENT DESCENT...
# WE SUGGEST A NETWORK WITH ONE HIDDEN LAYER WITH SIZE 200. 


S,X,allowed_a=env.Initialise_game()
N_a=np.shape(allowed_a)[0]   # TOTAL NUMBER OF POSSIBLE ACTIONS

N_in=np.shape(X)[0]    ## INPUT SIZE
N_h=200                ## NUMBER OF HIDDEN NODES


## INITALISE YOUR NEURAL NETWORK...
class QNetwork(nn.Module):
    """Actor (Policy) Model."""

    def __init__(self, N_in, N_h, N_a):
        """Initialize parameters and build model.
        Params
"""
        super(QNetwork, self).__init__()
        self.fc1 = nn.Linear(N_in, N_h)
        self.fc2 = nn.Linear(N_h, N_a)


    def forward(self, state):
        """Build a network that maps state -> action values."""
        x = F.relu(self.fc1(state))
        return self.fc2(x)


# HYPERPARAMETERS SUGGESTED (FOR A GRID SIZE OF 4)

epsilon_0 = 0.2     # STARTING VALUE OF EPSILON FOR THE EPSILON-GREEDY POLICY
beta = 0.00005      # THE PARAMETER SETS HOW QUICKLY THE VALUE OF EPSILON IS DECAYING (SEE epsilon_f BELOW)
gamma = 0.85        # THE DISCOUNT FACTOR
eta = 0.0035        # THE LEARNING RATE
update_every = 10

N_episodes = 100000 # THE NUMBER OF GAMES TO BE PLAYED 

# SAVING VARIABLES
R_save = np.zeros([N_episodes, 1])
N_moves_save = np.zeros([N_episodes, 1])


In [50]:
policy_n = QNetwork(N_in, N_h, N_a).float()
target_n = QNetwork(N_in, N_h, N_a).float()
criterion = torch.nn.MSELoss(reduction='sum')
optimizer = torch.optim.SGD(policy_n.parameters(), lr=1e-8, momentum=0.9)
memory = {"S":[], "X": [], "A":[], "R":[], "D": []}

In [ ]:
# TRAINING LOOP BONE STRUCTURE...
# I WROTE FOR YOU A RANDOM AGENT (THE RANDOM AGENT WILL BE SLOWER TO GIVE CHECKMATE THAN AN OPTIMISED ONE, 
# SO DON'T GET CONCERNED BY THE TIME IT TAKES), CHANGE WITH YOURS ...

for n in range(N_episodes):

    epsilon_f = epsilon_0 / (1 + beta * n)   ## DECAYING EPSILON
    Done=0                                   ## SET DONE TO ZERO (BEGINNING OF THE EPISODE)
    i = 1                                    ## COUNTER FOR NUMBER OF ACTIONS
    print(i)
    
    S,X,allowed_a=env.Initialise_game()      ## INITIALISE GAME
    #print(n)                                 ## REMOVE THIS OF COURSE, WE USED THIS TO CHECK THAT IT WAS RUNNING
    
    random_int = random.randint(0,1)
    
    while Done==0:                           ## START THE EPISODE
        
        
        ## THIS IS A RANDOM AGENT, CHANGE IT...
        print(epsilon_f)
        a,_=np.where(allowed_a==1)

        if epsilon_f > random_int:
            print("rand move")
            a_agent=np.random.permutation(a)[0]
        else:
            print("exploitation")
            policy_n.eval()
            with torch.no_grad():
                X = torch.from_numpy(X)
                a_agent = policy_n(X.float())
                a_agent = a_agent.numpy()
                a_agent = [a_agent[i] for i in a]
                a_agent = np.argmax(a_agent)
                a_agent = a[a_agent]
            policy_n.train()
                
    

        S_next,X_next,allowed_a_next,R,Done=env.OneStep(a_agent)
        memory["S"].append(S_next)
        memory["X"].append(X_next)
        memory["A"].append(allowed_a_next)
        memory["R"].append(R)
        memory["D"].append(Done)
        
        
        
        ## THE EPISODE HAS ENDED, UPDATE...BE CAREFUL, THIS IS THE LAST STEP OF THE EPISODE
        if Done==1 or i > update_every:
            print("Im here")
            
            break
        
        
        
        
        # IF THE EPISODE IS NOT OVER...
        else:
            
            ## ONLY TO PUT SUMETHING
            PIPPO=1
            
            
        # NEXT STATE AND CO. BECOME ACTUAL STATE...     
        S=np.copy(S_next)
        X=np.copy(X_next)
        allowed_a=np.copy(allowed_a_next)
        
        i += 1  # UPDATE COUNTER FOR NUMBER OF ACTIONS
        

        
        
    
    

1
0.2
rand move
0.2
rand move
0.2
rand move
0.2
rand move
Im here
1
0.19999000049997498
exploitation
Im here
1
0.19998000199980004
rand move
Im here
1
0.1999700044993251
rand move
Im here
1
0.19996000799840033
exploitation
0.19996000799840033
exploitation
0.19996000799840033
exploitation
Im here
1
0.19995001249687577
rand move
0.19995001249687577
rand move
0.19995001249687577
rand move
0.19995001249687577
rand move
0.19995001249687577
rand move
0.19995001249687577
rand move
Im here
1
0.19994001799460165
exploitation
0.19994001799460165
exploitation
0.19994001799460165
exploitation
0.19994001799460165
exploitation
0.19994001799460165
exploitation
0.19994001799460165
exploitation
0.19994001799460165
exploitation
0.19994001799460165
exploitation
Im here
1
0.199930024491428
exploitation
0.199930024491428
exploitation
0.199930024491428
exploitation
0.199930024491428
exploitation
Im here
1
0.19992003198720515
rand move
0.19992003198720515
rand move
Im here
1
0.1999100404817832
exploitation
0

0.19932230416583616
rand move
0.19932230416583616
rand move
0.19932230416583616
rand move
0.19932230416583616
rand move
0.19932230416583616
rand move
0.19932230416583616
rand move
0.19932230416583616
rand move
Im here
1
0.1993123723155115
exploitation
Im here
1
0.19930244145490783
exploitation
Im here
1
0.19929251158387726
rand move
Im here
1
0.1992825827022718
exploitation
Im here
1
0.19927265480994372
rand move
Im here
1
0.19926272790674504
exploitation
0.19926272790674504
exploitation
0.19926272790674504
exploitation
0.19926272790674504
exploitation
0.19926272790674504
exploitation
0.19926272790674504
exploitation
0.19926272790674504
exploitation
0.19926272790674504
exploitation
0.19926272790674504
exploitation
0.19926272790674504
exploitation
0.19926272790674504
exploitation
Im here
1
0.19925280199252804
exploitation
Im here
1
0.19924287706714486
exploitation
0.19924287706714486
exploitation
0.19924287706714486
exploitation
0.19924287706714486
exploitation
0.19924287706714486
explo

0.1988071570576541
exploitation
0.1988071570576541
exploitation
0.1988071570576541
exploitation
0.1988071570576541
exploitation
0.1988071570576541
exploitation
0.1988071570576541
exploitation
Im here
1
0.19879727647731224
exploitation
Im here
1
0.19878739687903788
rand move
0.19878739687903788
rand move
Im here
1
0.19877751826268447
rand move
Im here
1
0.19876764062810576
rand move
0.19876764062810576
rand move
0.19876764062810576
rand move
Im here
1
0.19875776397515527
rand move
0.19875776397515527
rand move
0.19875776397515527
rand move
0.19875776397515527
rand move
0.19875776397515527
rand move
0.19875776397515527
rand move
0.19875776397515527
rand move
0.19875776397515527
rand move
0.19875776397515527
rand move
0.19875776397515527
rand move
0.19875776397515527
rand move
Im here
1
0.19874788830368678
rand move
0.19874788830368678
rand move
0.19874788830368678
rand move
0.19874788830368678
rand move
0.19874788830368678
rand move
0.19874788830368678
rand move
0.19874788830368678
rand 

0.19837333862328904
exploitation
Im here
1
0.19836350111579468
exploitation
0.19836350111579468
exploitation
0.19836350111579468
exploitation
0.19836350111579468
exploitation
0.19836350111579468
exploitation
0.19836350111579468
exploitation
0.19836350111579468
exploitation
0.19836350111579468
exploitation
0.19836350111579468
exploitation
0.19836350111579468
exploitation
0.19836350111579468
exploitation
Im here
1
0.1983536645839532
exploitation
0.1983536645839532
exploitation
0.1983536645839532
exploitation
0.1983536645839532
exploitation
0.1983536645839532
exploitation
0.1983536645839532
exploitation
0.1983536645839532
exploitation
0.1983536645839532
exploitation
0.1983536645839532
exploitation
0.1983536645839532
exploitation
0.1983536645839532
exploitation
Im here
1
0.19834382902761938
exploitation
0.19834382902761938
exploitation
0.19834382902761938
exploitation
0.19834382902761938
exploitation
0.19834382902761938
exploitation
0.19834382902761938
exploitation
0.19834382902761938
expl

Im here
1
0.19790223629527015
exploitation
Im here
1
0.1978924454558947
rand move
0.1978924454558947
rand move
0.1978924454558947
rand move
Im here
1
0.197882655585238
rand move
0.197882655585238
rand move
0.197882655585238
rand move
0.197882655585238
rand move
0.197882655585238
rand move
Im here
1
0.19787286668315607
rand move
0.19787286668315607
rand move
0.19787286668315607
rand move
0.19787286668315607
rand move
0.19787286668315607
rand move
0.19787286668315607
rand move
0.19787286668315607
rand move
0.19787286668315607
rand move
0.19787286668315607
rand move
0.19787286668315607
rand move
0.19787286668315607
rand move
Im here
1
0.19786307874950537
exploitation
0.19786307874950537
exploitation
0.19786307874950537
exploitation
0.19786307874950537
exploitation
0.19786307874950537
exploitation
0.19786307874950537
exploitation
0.19786307874950537
exploitation
0.19786307874950537
exploitation
Im here
1
0.19785329178414207
exploitation
0.19785329178414207
exploitation
0.19785329178414207


...

0.16469038208168646
rand move
0.16469038208168646
rand move
0.16469038208168646
rand move
0.16469038208168646
rand move
0.16469038208168646
rand move
0.16469038208168646
rand move
0.16469038208168646
rand move
0.16469038208168646
rand move
0.16469038208168646
rand move
Im here
1
0.16468360163036766
exploitation
0.16468360163036766
exploitation
0.16468360163036766
exploitation
0.16468360163036766
exploitation
0.16468360163036766
exploitation
0.16468360163036766
exploitation
0.16468360163036766
exploitation
0.16468360163036766
exploitation
0.16468360163036766
exploitation
0.16468360163036766
exploitation
0.16468360163036766
exploitation
Im here
1
0.16467682173734047
rand move
0.16467682173734047
rand move
0.16467682173734047
rand move
0.16467682173734047
rand move
0.16467682173734047
rand move
0.16467682173734047
rand move
0.16467682173734047
rand move
0.16467682173734047
rand move
0.16467682173734047
rand move
0.16467682173734047
rand move
0.16467682173734047
rand move
Im here
1
0.16467

0.164331785875683
rand move
0.164331785875683
rand move
0.164331785875683
rand move
Im here
1
0.1643250349190699
exploitation
0.1643250349190699
exploitation
0.1643250349190699
exploitation
Im here
1
0.16431828451710967
exploitation
0.16431828451710967
exploitation
0.16431828451710967
exploitation
0.16431828451710967
exploitation
0.16431828451710967
exploitation
0.16431828451710967
exploitation
0.16431828451710967
exploitation
0.16431828451710967
exploitation
Im here
1
0.1643115346697338
exploitation
0.1643115346697338
exploitation
0.1643115346697338
exploitation
0.1643115346697338
exploitation
0.1643115346697338
exploitation
0.1643115346697338
exploitation
0.1643115346697338
exploitation
0.1643115346697338
exploitation
0.1643115346697338
exploitation
0.1643115346697338
exploitation
0.1643115346697338
exploitation
Im here
1
0.16430478537687412
rand move
Im here
1
0.16429803663846218
exploitation
0.16429803663846218
exploitation
0.16429803663846218
exploitation
0.16429803663846218
explo

0.16386726751331424
rand move
Im here
1
0.16386055466797755
rand move
0.16386055466797755
rand move
0.16386055466797755
rand move
0.16386055466797755
rand move
Im here
1
0.16385384237260364
rand move
0.16385384237260364
rand move
0.16385384237260364
rand move
0.16385384237260364
rand move
Im here
1
0.1638471306271249
rand move
0.1638471306271249
rand move
0.1638471306271249
rand move
0.1638471306271249
rand move
Im here
1
0.16384041943147376
exploitation
Im here
1
0.16383370878558265
rand move
0.16383370878558265
rand move
0.16383370878558265
rand move
0.16383370878558265
rand move
0.16383370878558265
rand move
0.16383370878558265
rand move
0.16383370878558265
rand move
0.16383370878558265
rand move
0.16383370878558265
rand move
0.16383370878558265
rand move
0.16383370878558265
rand move
Im here
1
0.163826998689384
exploitation
0.163826998689384
exploitation
0.163826998689384
exploitation
0.163826998689384
exploitation
0.163826998689384
exploitation
0.163826998689384
exploitation
0.163

0.16352561219901068
rand move
0.16352561219901068
rand move
Im here
1
0.1635189273158368
exploitation
0.1635189273158368
exploitation
0.1635189273158368
exploitation
0.1635189273158368
exploitation
0.1635189273158368
exploitation
0.1635189273158368
exploitation
0.1635189273158368
exploitation
0.1635189273158368
exploitation
Im here
1
0.1635122429791931
rand move
0.1635122429791931
rand move
0.1635122429791931
rand move
0.1635122429791931
rand move
0.1635122429791931
rand move
0.1635122429791931
rand move
0.1635122429791931
rand move
0.1635122429791931
rand move
Im here
1
0.16350555918901244
rand move
0.16350555918901244
rand move
Im here
1
0.16349887594522788
exploitation
0.16349887594522788
exploitation
Im here
1
0.16349219324777242
rand move
0.16349219324777242
rand move
0.16349219324777242
rand move
0.16349219324777242
rand move
0.16349219324777242
rand move
0.16349219324777242
rand move
0.16349219324777242
rand move
0.16349219324777242
rand move
0.16349219324777242
rand move
0.1634

Im here
1
0.16315875346712352
rand move
0.16315875346712352
rand move
0.16315875346712352
rand move
0.16315875346712352
rand move
Im here
1
0.1631520985438675
exploitation
0.1631520985438675
exploitation
0.1631520985438675
exploitation
Im here
1
0.16314544416347174
rand move
Im here
1
0.16313879032586973
exploitation
Im here
1
0.1631321370309951
rand move
0.1631321370309951
rand move
0.1631321370309951
rand move
0.1631321370309951
rand move
0.1631321370309951
rand move
0.1631321370309951
rand move
0.1631321370309951
rand move
Im here
1
0.16312548427878146
rand move
0.16312548427878146
rand move
0.16312548427878146
rand move
0.16312548427878146
rand move
0.16312548427878146
rand move
0.16312548427878146
rand move
0.16312548427878146
rand move
0.16312548427878146
rand move
0.16312548427878146
rand move
0.16312548427878146
rand move
0.16312548427878146
rand move
Im here
1
0.1631188320691624
rand move
0.1631188320691624
rand move
0.1631188320691624
rand move
0.1631188320691624
rand move
Im